Event Processor
===============
The EventLoop is the inner object that executes all of the code for a single loop for a single bar. It is controlled by Runner
the outer loop.

In [1]:
import pandas as pd
from collections import namedtuple
from montauk.database import tapdb
import montauk.database.utils as dbutils
import puma as tw
import data as datalib


In [2]:
# setup database credentials
test_login = credentials('test', 'localhost')
db_credentials = credentials('test', 'localhost', prefix='db_')

# setup temp database and connections
prod_tapdb = tapdb.tapdb_engine(**test_login)
temp_tapdb = dbutils.make_engine('temp_tapdb', **test_login)
dbutils.copy_table_schema(prod_tapdb, temp_tapdb)
dbutils.copy_table_data(prod_tapdb, temp_tapdb, include_tables=['source'])

# setup environment objects
oms = tw.order_manager.OrderManager('test_unit', temp_tapdb)
tap = tw.PositionManager('test_unit', oms, temp_tapdb)
port = tw.Portfolio('port_test', oms, tap)
risk = tw.Risk(oms)

In [3]:
# setup paper broker and paper exchange

exchange = tw.exchange.PaperExchange(live_frequency='1min')
broker = tw.PaperBroker('broker_01', oms, exchange)

In [4]:
# setup market data

datafeed = datalib.CsvDataFeed('c:/rmbaries/git/pifq/src/montauk/data/tests/inst/csv_data_feed')
hmds = datalib.HistoricalDataManager(datafeed, **db_credentials)
lmds = datalib.LiveDataManager(datafeed, **db_credentials)
mdm = datalib.MarketDataManager(hmds, lmds)
tap.setup_market_data(mdm)

In [5]:
# setup strategy
obj_bridge = namedtuple('OB', 'order_manager, market_data_manager')(oms, mdm)

strat = tw.strategy.ExampleStrategy('test.example', obj_bridge)
strat.add_symbols([('stock', 'test.sym.3', '1min'), ('stock', 'AAPL', '1min'), ('stock', 'MSFT', '1min')])
port.add_strategy(strat)

In [6]:
# setup the EventProcessor
event_loop = tw.EventProcessor([strat], [port], risk, oms, tap, broker, mdm, exchange)

In [7]:
# start the strategies
strat.start()

In [8]:
# process the first bar
mdm.bartime = pd.Timestamp('2010-01-01 09:30:00', tz='America/New_York')
event_loop.begin_of_day()
event_loop.market_open(['stock'])
event_loop.process_bar(['stock'], '1min')

In [9]:
# that process will have created orders and sent to broker and then exchange.
# Open orders in OMS. Note that the orders are SENT because the arrived at the exchnage after the exchange had processed orders
# for that bar.
print(oms.open_orders_df())

In [10]:
# and the closed orders
print(oms.closed_orders_df())

In [11]:
# Now process the next bar
mdm.bartime = '2010-01-01 09:31:00'  # passing a string uses default_time_zone
event_loop.process_bar(['stock'], '1min')

# See that the previous open orders are now in LIVE state
print(oms.open_orders_df())

In [12]:
# and that one order was filled and moved to closed state
print(oms.closed_orders_df())

In [13]:
# see the closed orders in the PositionManager
print(tap.positions_df)

In [14]:
# at any time can check to make sure there are no stuck orders along the path
event_loop.check_stuck_orders()

In [15]:
# run the end of day process
event_loop.market_close(['stock'])
event_loop.end_of_day(['stock'])

In [16]:
# see that the positions_df has been saved to the database
print(tapdb.get_positions_df(temp_tapdb, 'test_unit', mdm.bartime))

In [17]:
# see the orders saved to database
print(tapdb.get_orders_df(temp_tapdb, 'test_unit', mdm.bartime))